In [9]:
import os

In [ ]:
SCRIPT_DIR = os.path.dirname(os.path.abspath('/nlsasfs/home/gpucbh/vyakti11/iitrpr11/tasks'))

DATA_DIR = os.path.join(SCRIPT_DIR, 'data/Train')

DATA_DIR = os.path.normpath(DATA_DIR)

In [ ]:
import sys
import cv2
import numpy as np
import pandas as pd
from scipy.signal import detrend

def CHROM_METHOD(signal, fps):
    """
    Chrominance-based (CHROM) rPPG Algorithm.
    
    Args:
        signal (np.array): Input RGB signal with shape (Frames, 3)
        fps (float): Frames per second
        
    Returns:
        np.array: The extracted rPPG waveform
    """
    C = signal.T 

    mean_C = np.mean(C, axis=1, keepdims=True)
    C_norm = C / mean_C

    # 2. Define the Chrominance signals
    # Xs = 3*R - 2*G
    # Ys = 1.5*R + G - 1.5*B
    X_s = 3 * C_norm[0, :] - 2 * C_norm[1, :]
    Y_s = 1.5 * C_norm[0, :] + C_norm[1, :] - 1.5 * C_norm[2, :]

    # 3. Bandpass Filtering (Optional but recommended in CHROM)
    # apply detrending here to keep it consistent with your original script
    X_s = detrend(X_s)
    Y_s = detrend(Y_s)

    # 4. Signal Combination
    # Alpha = std(Xs) / std(Ys)
    # Final Signal = Xs - alpha * Ys
    alpha = np.std(X_s) / np.std(Y_s)
    h = X_s - alpha * Y_s

    return h

DATA_DIR = os.path.normpath(DATA_DIR)
META_FILE = os.path.join(DATA_DIR, "labels_train.xlsx")

print(f"Script Location: {SCRIPT_DIR}")
print(f"Data Location:   {DATA_DIR}")

if not os.path.exists(DATA_DIR):
    print(f"Error: The directory '{DATA_DIR}' does not exist.")
    sys.exit()

# ---- 3. FACE DETECTION (Unchanged) ----
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def extract_face(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0: return None
    x, y, w, h = max(faces, key=lambda b: b[2] * b[3])
    return frame[y:y+h, x:x+w]

def extract_rgb_signal(video_path):
    cap = cv2.VideoCapture(video_path)
    rgb_signals = []
    if not cap.isOpened(): return np.array([])
    while True:
        ret, frame = cap.read()
        if not ret: break
        face = extract_face(frame)
        if face is None: continue
        try:
            face = cv2.resize(face, (128, 128))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            mean_rgb = np.mean(face.reshape(-1, 3), axis=0)
            rgb_signals.append(mean_rgb)
        except: continue
    cap.release()
    return np.array(rgb_signals)

def main():
    if not os.path.exists(META_FILE):
        video_files = [f for f in os.listdir(DATA_DIR) if f.endswith(('.mp4', '.avi', '.mov'))]
        if not video_files: return
        df = pd.DataFrame({'video': video_files, 'label': [0] * len(video_files)})
    else:
        df = pd.read_excel(META_FILE)

    results = {}
    for _, row in df.iterrows():
        video_name = row["video"]
        label = row["label"]
        video_path = os.path.join(DATA_DIR, video_name)
        print(f"Processing: {video_name}...")

        if not os.path.exists(video_path): continue

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()

        rgb_signal = extract_rgb_signal(video_path)
        if len(rgb_signal) < fps: continue

        try:
            rppg_waveform = CHROM_METHOD(rgb_signal, fps)
            results[video_name] = {"label": label, "rppg": rppg_waveform}
            print(f"Success: {video_name} | Length: {len(rppg_waveform)}")
        except Exception as e:
            print(f"Error: {e}")

    output_path = os.path.join(DATA_DIR, "rppg_waveforms.npy")
    if results:
        np.save(output_path, results)
        print(f"\nSaved to: {output_path}")

if __name__ == "__main__":
    main()

📂 Script Location: /nlsasfs/home/gpucbh/vyakti11/iitrpr11
📂 Data Location:   /nlsasfs/home/gpucbh/vyakti11/iitrpr11/data/Train
Processing: subject_10_Vid_6.avi...
Processing: subject_11_Vid_6.MP4...
Processing: subject_12_Vid_6.MP4...
Processing: subject_15_Vid_6.avi...
Processing: subject_16_Vid_6.MP4...
Processing: subject_18_Vid_6.webm...
Processing: subject_1_Vid_5.avi...
Processing: subject_20_Vid_2.avi...
Success: subject_20_Vid_2.avi | Length: 2719
Processing: subject_20_Vid_5_2.avi...
Processing: subject_20_Vid_6.avi...
Success: subject_20_Vid_6.avi | Length: 2407
Processing: subject_20_Vid_7.avi...
Processing: subject_21_Vid_5.avi...
Processing: subject_23_Vid_5.avi...
